In [19]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = 100

In [20]:
data = pd.read_csv('input/train.csv')

In [21]:
data.shape

(10000, 20)

In [22]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


### train_test_split

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [25]:
train.shape, valid.shape

((7000, 20), (3000, 20))

### Prepare train, valid

In [26]:
distr_info1 = train['DistrictId'].value_counts(normalize=True).reset_index().rename(columns={'index':'DistrictId', 
                                                                               'DistrictId':'flat_qty_distr'})

In [27]:
def clean_rooms(df, source_df):
    df.loc[df['Rooms'] > 6, 'Rooms'] = source_df['Rooms'].median()
    return df

In [28]:
def clean_square(df):
    df.loc[df['Square'] < 15, 'Square'] = 15
    df.loc[df['KitchenSquare'] < 5, 'KitchenSquare'] = 5
    return df

In [29]:
def clean_year(df):
    df.loc[df['HouseYear'] > 2020, 'HouseYear'] = 2020
    return df

In [30]:
def add_district_info1(df, distr_info1):
    df = pd.merge(df, distr_info1, on='DistrictId', how='left')
    df['flat_qty_distr'] = df['flat_qty_distr'].fillna(0.000143)
    return df

In [31]:
distr_stat_dr = train.groupby(['DistrictId', 'Rooms'], as_index=False)[['Price']].median().\
    rename(columns={'Price':'median_price_dr'})
    
distr_stat_r = train.groupby(['Rooms'], as_index=False)[['Price']].median().rename(columns={'Price':'median_price_r'})

median_price = train['Price'].median()

In [32]:
def add_stats(df, distr_stat_dr, distr_stat_r, median_price):
    df = pd.merge(df, distr_stat_dr, on=['DistrictId', 'Rooms'], how='left')
    df = pd.merge(df, distr_stat_r, on='Rooms', how='left')
    
    df['median_price_r'] = df['median_price_r'].fillna(int(median_price))
    df['median_price_dr'] = df['median_price_dr'].fillna(df['median_price_r'])
    return df

In [33]:
def add_cat_fts(df, cat_fts=('Ecology_2', 'Ecology_3', 'Shops_2')):
    for col in cat_fts:
        df[col] = (df[col] == 'B').astype(int)
    return df

In [34]:
def fillna_healthcare_1(df):
    df['Healthcare_1'] = df['Healthcare_1'].fillna(0)
    return df

In [35]:
def prepare_data(df, distr_info1, distr_stat_dr, distr_stat_r, median_price, source_df = train):
    df = clean_rooms(df, source_df)
    df = clean_square(df)
    df = clean_year(df)
    df = add_district_info1(df, distr_info1)
    df = add_stats(df, distr_stat_dr, distr_stat_r, median_price)
    df = add_cat_fts(df)
    df = fillna_healthcare_1(df)
    return df

In [36]:
train = prepare_data(train, distr_info1, distr_stat_dr, distr_stat_r, median_price)

In [37]:
train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,flat_qty_distr,median_price_dr,median_price_r
0,14604,23,1.0,41.681380,22.796166,8.0,14,17.0,2015,0.075779,1,1,6,1437,3,0.0,0,2,1,88504.384965,0.056286,105350.440609,155464.088358
1,5621,23,3.0,163.495333,161.504222,12.0,5,3.0,1977,0.014073,1,1,2,475,0,0.0,0,0,1,207007.956663,0.056286,157606.065847,269162.365906
2,235,87,1.0,39.710131,19.538663,8.0,4,17.0,1986,0.100456,1,1,43,7227,0,0.0,1,6,0,182126.280899,0.003000,178279.774392,155464.088358
3,16258,48,3.0,96.056784,98.152802,5.0,15,1.0,2017,0.041125,1,1,46,9515,5,0.0,1,10,1,524365.550705,0.008857,397380.028528,269162.365906
4,10773,77,3.0,79.195825,44.421062,10.0,16,17.0,1984,0.298205,1,1,16,4048,3,0.0,1,3,1,322048.433990,0.004000,252818.703924,269162.365906


In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 23 columns):
Id                 7000 non-null int64
DistrictId         7000 non-null int64
Rooms              7000 non-null float64
Square             7000 non-null float64
LifeSquare         5514 non-null float64
KitchenSquare      7000 non-null float64
Floor              7000 non-null int64
HouseFloor         7000 non-null float64
HouseYear          7000 non-null int64
Ecology_1          7000 non-null float64
Ecology_2          7000 non-null int32
Ecology_3          7000 non-null int32
Social_1           7000 non-null int64
Social_2           7000 non-null int64
Social_3           7000 non-null int64
Healthcare_1       7000 non-null float64
Helthcare_2        7000 non-null int64
Shops_1            7000 non-null int64
Shops_2            7000 non-null int32
Price              7000 non-null float64
flat_qty_distr     7000 non-null float64
median_price_dr    7000 non-null float64
median_price_r 

In [39]:
valid = prepare_data(valid, distr_info1, distr_stat_dr, distr_stat_r, median_price)

### Model

In [40]:
train.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2', 'Price', 'flat_qty_distr',
       'median_price_dr', 'median_price_r'],
      dtype='object')

In [41]:
feats = ['Rooms', 'Square', 'Social_1','median_price_dr', 'Helthcare_2', 'Healthcare_1', 'flat_qty_distr',
         'Floor','HouseFloor', 'HouseYear', 'Ecology_1','Social_2', 'Social_3', 'Shops_2', 'KitchenSquare' ]


In [42]:
from sklearn.ensemble import RandomForestRegressor as RF

In [43]:
model = RF(n_estimators=420, max_depth=14, random_state= 42, max_features=2)


In [44]:
model.fit(train.loc[:, feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=14,
           max_features=2, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=420, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [45]:
model.feature_importances_

array([0.11077039, 0.19132318, 0.04833061, 0.24532475, 0.02472167,
       0.0222504 , 0.06758508, 0.03260005, 0.03656947, 0.04669396,
       0.02906428, 0.05033642, 0.04114196, 0.00145452, 0.05183325])

### Evaluate model

In [46]:
from sklearn.metrics import r2_score as r2

In [47]:
def evaluate_model(df, model=model, feats=feats, check_r2='Yes'):
    pred = model.predict(df.loc[:, feats])
    if check_r2 == 'Yes':
        r2_ = r2(df['Price'], pred)
        print('R2: {}'.format(r2_))
    return pred

In [48]:
pred_train = evaluate_model(train)

R2: 0.9306408283133625


In [49]:
pred_valid = evaluate_model(valid)

R2: 0.724951332950533


### Test

In [50]:
test = pd.read_csv('input/test.csv')

In [51]:
test = prepare_data(test, distr_info1, distr_stat_dr, distr_stat_r, median_price)

In [52]:
test['Price'] = model.predict(test.loc[:, feats])

In [53]:
test.loc[:, ['Id', 'Price']].to_csv('LBogdanov_predictions.csv', index=None)